In [1]:
from skimage.io import imread, imshow, imsave
import os
import numpy as np
import stackview
from matplotlib import pyplot as plt
from skimage import data, io
from skimage import morphology
from skimage import filters
from skimage import measure
import pandas as pd
from glob import glob
from datetime import datetime
current_dateTime = datetime.now()
import warnings
warnings.filterwarnings('ignore')


def ini_params(images, imageb, Pixel_Cutoff:int=0, Blank_Percentile:float=0.0):

    blank_image = np.clip(imageb, Pixel_Cutoff, 65535)
    blank_image[blank_image <= Pixel_Cutoff] = 0
    
    signal_image = images - (np.minimum(images, blank_image * Blank_Percentile))
    
    return signal_image

def make_mask(blank_im, disk1:int=4, disk2:int=1):
    blank_smoothed = morphology.white_tophat(blank_im, morphology.disk(disk1))
    blank_threshold = filters.threshold_otsu(blank_smoothed)
    autofluorescent_mask = blank_im > blank_threshold

    autofluorescent_mask = morphology.binary_closing(autofluorescent_mask, morphology.disk(disk2))
    return autofluorescent_mask.astype(np.uint8)

def find_gauss(signal_image2, sigma_value:int=10):
    signal_image = signal_image2 * 1.01
    blank = filters.gaussian(signal_image, sigma=sigma_value, preserve_range=True)
    return blank

def clipped_blank(blank_image, Gauss_low:int = 0, Gauss_high:int = 65535):
    blank_image2 = blank_image * 1.01
    # blank2 = np.clip(blank_image, Gauss_low, np.max(blank_image))
    blank_image2[Gauss_low >= blank_image2] = 0  
    blank_image2[blank_image2 >= Gauss_high] = 0
    return blank_image2

def final_factor(sig_im, bl, low, high, Gauss_Percentile:float=0.2):

    bl[low >= bl] = 0  
    bl[bl >= high] = 0

    signal_im2 = sig_im - (np.minimum(sig_im, bl * Gauss_Percentile))
    return signal_im2

In [2]:
# Enter sample ID portion of filename:
sample_id = '20_06_N2_R1'

# Enter directory path:
path = 'Z:/Processed_CODEX/Misc/Intermediate_data/'

sample = sample_id + '_Processed'

# The single-marker images after registration in VALIS are in a folder named by sample ID and _Registered.  This creates folders for ImageJ-processed images, notebook-processed images and a folder for parameter .txt files. 
sample_base = sample.replace('_Registered', '_Processed')
os.makedirs(path + sample_base, exist_ok=True)
os.makedirs(path + sample_base + '/Processing_parameters', exist_ok=True)
os.makedirs(path + sample_base + '/ImageJ', exist_ok=True)

image_folder = path + sample_id + '_Registered/'
out_folder = path + sample_id + '_Processed/'

# seg_mask = imread(image_folder + sample_id +'_DAPI_segmentation.tif')

In [35]:
# This cell allows the user to choose which blank channel to use (BlankID) and determine how much, if any, blank image to subtract(Blank_Percentile).  It also allows for only subtracting a range of pixel values (Pixel_Cutoff).  Use these values in the next cell.
# For each marker, enter the name of the image file without the extension for signal_channel.  Then change the letter of bl_CH to match 
signal_channel = 'CD21'

# Do not change these
params_filename = out_folder + 'Processing_parameters/' + signal_channel + '_param.txt'
signal_image_tiff = imread(image_folder + signal_channel + '.tif')

# Change bl_CH to a, b, or c according to the channel that marker was imaged on i.e. if marker is CH3, the last letter of both blanks should be 'b', CH4 should be 'c'.  For CH2 markers, the blank channels often do not match the background autofluorescence well and can also not be suitable to use at all, so a suitable 'b' or 'c' blank should be chosen.
# Change bl_int to 13 or 1 to use a blank channel.  Blank 13 generally has photobleached autofluorescent signal compared with Blank 1, and generally removes more diffuse background.  Sometimes the images from cycles earlier in the run will match closer to blank1 while those later will match closer to blank13.

bl_CH = 'b'
bl_int = str(1)

# Do not change these
blankID = 'Blank' + bl_int + bl_CH
blank_image_tiff = imread(image_folder + blankID + '.tif')

# Sometimes blurring before subtraction can help.  Uncomment to apply.  Adjust sigma to control radius(strength) of blurring.
blank_image_tiff = filters.gaussian(blank_image_tiff, sigma=1, preserve_range=True)
# signal_image_tiff = filters.gaussian(signal_image_tiff, sigma=1, preserve_range=True)

# This prints the current marker channel selected for subtraction.
print(signal_channel)

#This first stackview.interact function is for viewing the effects of subtraction on a close-up portion of the image.  Adjust coordinates to determine the close-up; usually just the first digit of x2 and y2.  To assess subtraction, first slowly move the Blank_Percentile slider while noting if autofluorescent structures only are getting dimmer.  Continue until none of these remain while noting if marker signal is being diminished.  Then, slowly move the other slider until signal is brightest without reintroducing autofluorescence/noise.  Once values of Blank_Percentile and Pixel_Cutoff are found, use them for the next stackview.interact to see how they look on the entire image.

x1 = 2000
x2 = 7000
y1 = 2000
y2 = 7000

# Adjust zoom_factor to fit the image to your screen.  The other values can be adjusted as needed.
stackview.interact(ini_params, signal_image_tiff[x1:x2,y1:y2], blank_image_tiff[x1:x2,y1:y2], zoom_factor=0.4, colormap = 'viridis', min_value=0, max_value=50000, step=1000)

# Comment out the function above and uncomment the next function (CTRL + /) and check the values determined above.  Repeat this process as necessary.

# stackview.interact(ini_params, signal_image_tiff, blank_image_tiff, zoom_factor=0.1, colormap = 'viridis', min_value=0, max_value=50000, step=1000)

CD21


In [36]:
# This cell performs the subtraction with values determined above.  The lines that are commented out can be used to subtract another blank channel (blank_image_2) and perform gaussian blur.
# If no subtraction was necessary, enter 0 for both and then go to the save cell.

# Enter Pixel_Cutoff from above:
blank_clip_factor = 16000

# Enter Blank_Percentile from above:
background_scale_factor = 0.9

blank_image = np.clip(blank_image_tiff, blank_clip_factor, blank_image_tiff.max())
blank_image[blank_image <= blank_clip_factor] = 0

signal_image = signal_image_tiff - (np.minimum(signal_image_tiff, blank_image * background_scale_factor))

In [37]:
print(signal_channel)
bl_CH2 = 'c'
bl_int2 = str(13)

# Do not change these
blankID2 = 'Blank' + bl_int2 + bl_CH2
# blankID2 = 'CD15'
blank_image_tiff_2 = imread(image_folder + blankID2 + '.tif')
blank_image_tiff_2 = filters.gaussian(blank_image_tiff_2, sigma=1, preserve_range=True)

x1 = 2000
x2 = 9000
y1 = 2000
y2 = 9000

# Adjust zoom_factor to fit the image to your screen.  The other values can be adjusted as needed.
# stackview.interact(ini_params, signal_image[x1:x2,y1:y2], blank_image_tiff_2[x1:x2,y1:y2], zoom_factor=0.2, colormap = 'viridis', min_value=0, max_value=50000, step=1000)

# Comment out the function above and uncomment the next function (CTRL + /) and check the values determined above.  Repeat this process as necessary.
stackview.interact(ini_params, signal_image, blank_image_tiff_2, zoom_factor=0.1, colormap = 'viridis', min_value=0, max_value=50000, step=1000)

CD21


In [38]:
blank_clip_factor_2 = 9000
background_scale_factor_2 = 0.4
blank_image_2 = np.clip(blank_image_tiff_2, blank_clip_factor_2, blank_image_tiff_2.max())
blank_image_2[blank_image_2 <= blank_clip_factor_2] = 0
signal_image = signal_image - (np.minimum(signal_image, blank_image_2 * background_scale_factor_2))

In [ ]:
# If no other changes need to be made, go to the cells to view and save.
# Sometimes it is helpful to subtract the gaussian of an image to remove patterns of intensity.  This stackview.interact helps to find the optimal sigma value for blurring.
stackview.interact(find_gauss, signal_image, zoom_factor=0.1, colormap = 'viridis', min_value=0, max_value=60, step=5)

In [25]:
# Enter the sigma value to perform the blurring.
sigma = 20
blank = filters.gaussian(signal_image, sigma=sigma, preserve_range=True)

In [ ]:
# This cell allows the user to only apply a range of pixel values to the gaussian subtraction.  Run after performing the gaussian blur in the cell above.

# stackview.interact(clipped_blank, blank[x1:x2,y1:y2], zoom_factor=0.5, colormap = 'viridis', continuous_update=True, display_min = 0, display_max = None, min_value=0, max_value=np.max(blank), step=100)

stackview.interact(clipped_blank, blank, zoom_factor=0.1, colormap = 'viridis', continuous_update=True, display_min = 0, display_max = None, min_value=0, max_value=np.max(blank), step=100)

In [17]:
# This cell allows the user to determine how much subtraction to do, similar to Blank_Percentile above.  Enter the Gauss_Cutoff values from the cell above.

stackview.interact(final_factor, signal_image, blank,5500,14431, colormap = 'viridis', display_max = None, zoom_factor=.1)

# stackview.interact(final_factor, signal_image[x1:x2,y1:y2], blank[x1:x2,y1:y2], 0,4700, colormap = 'viridis', display_max = None, zoom_factor=.3)

In [74]:
# This cell performs the gaussian subtraction.  Note that the image is renamed from signal_image to signal_image2 so saving and viewing either image can be changed by adding/removing the '2' below.

# Enter Gauss_Percentile for s_image_factor:
s_image = signal_image
s_image_factor = 0.4

# Enter Gauss_Cutoff for blank_factor:
g_low = 5300
g_high = 15528
blank[g_low >= blank] = 0  
blank[blank >= g_high] = 0

signal_image2 = s_image - (np.minimum(s_image, blank * s_image_factor))

In [ ]:
# This cell is for checking the image before saving.  Add or remove the '2' after signal_image to view the first or second (typically gaussian subtracted) image.

fig, axes = plt.subplots(1, 1, figsize=(25, 25))
im = axes.imshow(signal_image,)
fig.colorbar(im, ax=axes, shrink=.65)
plt.show()

In [39]:
# Even with this process, it may be that certain markers should be excluded.  Manually create a "failed markers" file to list these and save it in the 'Processing_parameters' folder.
# This cell saves the image and the parameters used.  Add or remove the '2' as necessary.  The next steps in FIJI should be saved as well.  For now, FIJI: Plugins>Macros>Record to record all steps and copy/paste to params file.

imsave(out_folder + signal_channel + '.tif', signal_image)

# Anything here will be added to .txt file:
Notes = '''

'''

# Get all variables in the global scope
variables = globals()
variable_names_to_save = ['current_dateTime', 'signal_channel', 'blankID', 'blank_clip_factor', 'background_scale_factor', 'blankID2', 'blank_clip_factor_2', 'background_scale_factor_2', 'sigma', 'g_low', 'g_high', 's_image_factor', 'blank_factor', 'Notes']

# Open the file in write mode
with open(params_filename, 'w') as file:
    # Iterate over each variable name to save
    for var_name in variable_names_to_save:
        # Check if the variable exists in the global scope
        if var_name in variables:
            # Get the variable value
            var_value = variables[var_name]
            # Convert the variable to a string representation
            var_str = f'{var_name}: {repr(var_value)}\n'
            # Write the variable string to the file
            file.write(var_str)

In [7]:
# This cell merges all the images together.  May have to input pixel sizes manually. 

# Enter sample ID portion of filename:
sample_id = '20_06_N1_R2'

# Enter directory path:
path = 'Z:/Processed_CODEX/HubMap_LymphNode/'

sample = sample_id + '_Processed/'

folder = sample + 'ImageJ/'

pixelsize = 0.3774
channel_names = []

for file in glob(path +sample+'*.tif'):

    file_name = os.path.basename(file)
    # print(file_name)
    file_name = file_name.split('.')
    channel_name = file_name[0]
    # print(channel_name)
    channel_names.append(channel_name)

image = imread(path +sample+'*.tif').astype(np.uint16)

imsave('C:/Users/smith6jt/QuPath_Projects/Lymph_Node/Images/'+sample_id+'.tif', image, imagej=True, resolution=(1/pixelsize, 1/pixelsize), resolutionunit='MICROMETER', metadata={'axes': 'CYX', 'Labels': channel_names})

In [ ]:
# This function may be used in place of ini_params above to assess how subtraction affects signal to noise.  Must have a binary segmentation file.
# def ini_params_SNR(images, imageb, seg_mask, blclfa:int=0, bckscfa:float=.0):

#     blank_image = np.clip(imageb, blclfa, 65535)
#     blank_image[blank_image <= blclfa] = 0

#     signal_image = images - (np.minimum(images, blank_image * bckscfa))
#     seg_mask = measure.label(seg_mask, return_num=True, background=0)
#     properties = measure.regionprops(seg_mask[0], intensity_image=signal_image)
#     statistics = {
   
#     'area':       [p.area               for p in properties if p.area<800],
#     'mean':       [p.mean_intensity     for p in properties if p.area<800]
#     }
#     df = pd.DataFrame(statistics)
    
#     MFI = np.asarray(df['mean'])
#     aX = MFI.flatten()
#     # compute 20 largest values in aX
#     top20 = np.sort(aX)[-20:]
#     # compute the mean of bottom 10th percentile of aX
#     btm10 = np.sort(aX)[:int(len(aX)*0.1)]
#     top20btm10 = np.mean(top20)/np.mean(btm10)
    
#     print('SNR is ' + str(top20btm10)) 
#     print(df.describe())
#     return signal_image